### Import necessary Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Libraries imported.')

Libraries imported.


### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'KUP5WIDELDBI3TU4A5WPREGM0TO2XKHPOGL1DF4J3PYFZQ3T' # your Foursquare ID
CLIENT_SECRET = 'E5JCI2UZBNSXHSI1UI113UOUX2MHYVZN0ZRQMHWY5RAMSSXL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [3]:
address = '1755 W Jackson Blvd, Chicago, IL 60612'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.8773912 -87.6712330997087


In [4]:
search_query = 'Restaurant'
radius = 500
print(search_query + ' .... OK!')

Restaurant .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=KUP5WIDELDBI3TU4A5WPREGM0TO2XKHPOGL1DF4J3PYFZQ3T&client_secret=E5JCI2UZBNSXHSI1UI113UOUX2MHYVZN0ZRQMHWY5RAMSSXL&ll=41.8773912,-87.6712330997087&v=20180604&query=Restaurant&radius=500&limit=30'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c9b5c124434b9611a5c3de8'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1c1941735',
      'name': 'Mexican Restaurant',
      'pluralName': 'Mexican Restaurants',
      'primary': True,
      'shortName': 'Mexican'}],
    'hasPerk': False,
    'id': '4bb3822d14cfd13a192116ab',
    'location': {'address': '230 S Ashland Ave',
     'cc': 'US',
     'city': 'Chicago',
     'country': 'United States',
     'crossStreet': 'at W Jackson Blvd',
     'distance': 350,
     'formattedAddress': ['230 S Ashland Ave (at W Jackson Blvd)',
      'Chicago, IL 60607',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.87793,
       'lng': -87.66707}],
     'lat': 41.87793,
     'lng': -87.66707,
     'postalCode': '60607',
     'state': 'IL'},
    'name': "Angel's Restaurant",
    'referralId': 'v-155368552

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'shortName': 'Mexican', 'name': 'Mexican Res...",False,4bb3822d14cfd13a192116ab,230 S Ashland Ave,US,Chicago,United States,at W Jackson Blvd,350,"[230 S Ashland Ave (at W Jackson Blvd), Chicag...","[{'lng': -87.66707, 'lat': 41.87793, 'label': ...",41.877930,-87.667070,60607,IL,Angel's Restaurant,v-1553685522
1,"[{'shortName': 'American', 'name': 'American R...",False,4c6044a347429c7455ce217a,1908 W Harrison St,US,Chicago,United States,NaN,436,"[1908 W Harrison St, Chicago, IL 60612, United...","[{'lng': -87.67420984094747, 'lat': 41.8741526...",41.874153,-87.674210,60612,IL,Red Castle Restaurant,v-1553685522
2,"[{'shortName': 'Chinese', 'name': 'Chinese Res...",False,4f324e4019836c91c7caaa91,2008 W Madison St,US,Chicago,United States,NaN,645,"[2008 W Madison St, Chicago, IL, United States]","[{'lng': -87.67699, 'lat': 41.881294, 'label':...",41.881294,-87.676990,NaN,IL,China Dragon Restaurant,v-1553685522
3,"[{'shortName': 'Building', 'name': 'Building',...",False,4f32696219836c91c7d53ba6,1926 W Harrison St,US,Chicago,United States,NaN,435,"[1926 W Harrison St, Chicago, IL 60612, United...","[{'lng': -87.67495443, 'lat': 41.87462598, 'la...",41.874626,-87.674954,60612,IL,Medical Center Apartments,v-1553685522


#### Define information of interest and filter dataframe

In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Angel's Restaurant,Mexican Restaurant,230 S Ashland Ave,US,Chicago,United States,at W Jackson Blvd,350,"[230 S Ashland Ave (at W Jackson Blvd), Chicag...","[{'lng': -87.66707, 'lat': 41.87793, 'label': ...",41.877930,-87.667070,60607,IL,4bb3822d14cfd13a192116ab
1,Red Castle Restaurant,American Restaurant,1908 W Harrison St,US,Chicago,United States,NaN,436,"[1908 W Harrison St, Chicago, IL 60612, United...","[{'lng': -87.67420984094747, 'lat': 41.8741526...",41.874153,-87.674210,60612,IL,4c6044a347429c7455ce217a
2,China Dragon Restaurant,Chinese Restaurant,2008 W Madison St,US,Chicago,United States,NaN,645,"[2008 W Madison St, Chicago, IL, United States]","[{'lng': -87.67699, 'lat': 41.881294, 'label':...",41.881294,-87.676990,NaN,IL,4f324e4019836c91c7caaa91
3,Medical Center Apartments,Building,1926 W Harrison St,US,Chicago,United States,NaN,435,"[1926 W Harrison St, Chicago, IL 60612, United...","[{'lng': -87.67495443, 'lat': 41.87462598, 'la...",41.874626,-87.674954,60612,IL,4f32696219836c91c7d53ba6


In [10]:
dataframe_filtered.name

0           Angel's Restaurant
1        Red Castle Restaurant
2      China Dragon Restaurant
3    Medical Center Apartments
Name: name, dtype: object

In [11]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map